In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## Door knob

In [9]:
from pkg.utils.code_scraps import *

In [10]:
gscene.show_pose(crob.home_pose)

In [11]:
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(0.5,0.5,0.5), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=True, collision=True, color=(1,0,0,1))

In [12]:
add_lever(pscene, knob)

### prepare planner

In [13]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [14]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

   Use a production WSGI server instead.
 * Debug mode: off


### Plan

In [15]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['knob', 'lever_grip']
(0, 'knob')


In [16]:
goal_nodes = [(2,'knob')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 36/36 agents
try: 0 - (0, 'knob')->(1, 'knob')
try: 0 - (0, 'knob')->(0, 'grip0')
result: 0 - (0, 'knob')->(1, 'knob') = fail
try: 0 - (0, 'knob')->(1, 'knob')
try transition motion
try: 0 - (0, 'knob')->(0, 'grip0')
result: 0 - (0, 'knob')->(1, 'knob') = fail
try transition motion
try: 0 - (0, 'knob')->(1, 'knob')
result: 0 - (0, 'knob')->(1, 'knob') = fail
try: 0 - (0, 'knob')->(1, 'knob')
result: 0 - (0, 'knob')->(1, 'knob') = fail
try: 0 - (0, 'knob')->(1, 'knob')
result: 0 - (0, 'knob')->(1, 'knob') = fail
try: 0 - (0, 'knob')->(1, 'knob')
result: 0 - (0, 'knob')->(1, 'knob') = fail
transition motion tried: True
try: 0 - (0, 'knob')->(0, 'grip0')
try: 0 - (0, 'knob')->(0, 'grip0')
result: 0 - (0, 'knob')->(0, 'grip0') = success
branching: 0->1 (0.21/30.0 s, steps/err: 69(111.598968506 ms)/0.00049382099879)
Motion Filer Failure: ReachChecker
try: 0 - (0, 'knob')->(1, 'knob')
try transition motion
try: 0 - (0, 'knob')->(1, 'knob')
try: 0 - (0, 'knob')->(0, 'grip0')
result: 0 - (

try: 13 - (1, 'grip0')->(2, 'grip0')
result: 7 - (0, 'grip0')->(0, 'knob') = success
branching: 7->18 (0.72/30.0 s, steps/err: 1(5.52797317505 ms)/0)
try: 16 - (0, 'knob')->(1, 'knob')
try constrained motion
try: 18 - (0, 'knob')->(1, 'knob')
try: 10 - (0, 'knob')->(0, 'grip0')
result: 16 - (0, 'knob')->(1, 'knob') = fail
result: 18 - (0, 'knob')->(1, 'knob') = fail
try: 9 - (0, 'knob')->(0, 'grip0')
try: 2 - (1, 'grip0')->(2, 'grip0')
try transition motion
try constrained motion
try transition motion
try: 8 - (0, 'grip0')->(1, 'grip0')
result: 8 - (0, 'grip0')->(1, 'grip0') = success
transition motion tried: True
transition motion tried: True
branching: 8->19 (0.8/30.0 s, steps/err: 1(20.9760665894 ms)/0)
result: 9 - (0, 'knob')->(0, 'grip0') = success
branching: 9->20 (0.83/30.0 s, steps/err: 8(79.4048309326 ms)/0.00147437447756)
result: 10 - (0, 'knob')->(0, 'grip0') = success
try: 2 - (1, 'grip0')->(2, 'grip0')
branching: 10->21 (0.86/30.0 s, steps/err: 4(112.634181976 ms)/0.001945

try: 3 - (0, 'grip0')->(0, 'knob')
result: 2 - (1, 'grip0')->(2, 'grip0') = success
result: 5 - (0, 'grip0')->(1, 'grip0') = success
try: 39 - (2, 'grip0')->(2, 'knob')
result: 33 - (0, 'knob')->(1, 'knob') = fail
try: 31 - (1, 'grip0')->(2, 'grip0')
try: 34 - (0, 'grip0')->(0, 'knob')
transition motion tried: True
result: 28 - (1, 'grip0')->(2, 'grip0') = fail
Goal reached
branching: 2->39 (1.32/30.0 s, steps/err: 80(922.591924667 ms)/4.97738080239e-06)
branching: 23->36 (1.32/30.0 s, steps/err: 1(27.4920463562 ms)/0)
try: 12 - (0, 'knob')->(1, 'knob')
try transition motion
try transition motion
try: 37 - (1, 'grip0')->(2, 'grip0')
branching: 5->38 (1.32/30.0 s, steps/err: 1(40.009021759 ms)/0)
transition motion tried: True
try transition motion
try: 5 - (0, 'grip0')->(0, 'knob')
transition motion tried: True
try constrained motion
result: 34 - (0, 'grip0')->(1, 'grip0') = success
try: 23 - (0, 'grip0')->(0, 'knob')
try transition motion
result: 35 - (2, 'grip0')->(2, 'knob') = succes

try constrained motion
branching: 48->56 (1.54/30.0 s, steps/err: 1(5.60712814331 ms)/0)
result: 30 - (0, 'knob')->(0, 'grip0') = success
Motion Plan Failure
end
try: 38 - (1, 'grip0')->(2, 'grip0')
++ adding return motion to acquired answer ++
result: 57 - (0, 'knob')->(1, 'knob') = fail
try: 39 - (2, 'grip0')->(2, 'knob')
try: 5 - (0, 'grip0')->(0, 'knob')
branching: 30->58 (1.54/30.0 s, steps/err: 15(97.1088409424 ms)/0.00195915627658)
try constrained motion
try: 39 - (2, 'grip0')->(2, 'knob')
constrained motion tried: True
try constrained motion
end
try constrained motion
try transition motion
Goal reached
result: 49 - (0, 'knob')->(1, 'knob') = fail
try constrained motion
result: 7 - (0, 'grip0')->(1, 'grip0') = success
result: 39 - (2, 'grip0')->(2, 'knob') = success
constrained motion tried: True
result: 37 - (1, 'grip0')->(2, 'grip0') = fail
result: 2 - (1, 'grip0')->(2, 'grip0') = success
try: 61 - (2, 'grip0')->(2, 'knob')
Goal reached
result: 39 - (2, 'grip0')->(2, 'knob') =

Goal reached
Goal reached
branching: 65->83 (1.87/30.0 s, steps/err: 11(115.076780319 ms)/0.000707252264102)
try constrained motion
result: 49 - (0, 'knob')->(0, 'grip0') = success
++ adding return motion to acquired answer ++
try constrained motion
result: 13 - (1, 'grip0')->(2, 'grip0') = success
branching: 49->86 (1.88/30.0 s, steps/err: 98(222.47505188 ms)/0.00172057335857)
end
end
======================= terminated 24: required answers acquired  (1.9/30.0) ===============================
++ adding return motion to acquired answer ++
Goal reached
Goal reached
constrained motion tried: True
======================= terminated 11: Stop called from other agent  (1.9/30.0) ===============================
transition motion tried: True
branching: 13->87 (1.89/30.0 s, steps/err: 80(1156.88705444 ms)/2.97667961802e-05)
constrained motion tried: True
end
end
Goal reached
result: 2 - (1, 'grip0')->(2, 'grip0') = success
constrained motion tried: True
constrained motion tried: True
===========

======================= terminated 7: required answers acquired  (2.1/30.0) ===============================
======================= terminated 35: Stop called from other agent  (2.2/30.0) ===============================
end
constrained motion tried: True
result: 19 - (1, 'grip0')->(2, 'grip0') = success
branching: 19->101 (2.19/30.0 s, steps/err: 80(1050.60601234 ms)/2.97667961802e-05)
======================= terminated 21: Stop called from other agent  (2.2/30.0) ===============================
end
constrained motion tried: True
result: 2 - (1, 'grip0')->(2, 'grip0') = success
branching: 2->102 (2.21/30.0 s, steps/err: 80(1143.58115196 ms)/4.97738080239e-06)
======================= terminated 22: Stop called from other agent  (2.2/30.0) ===============================
end
constrained motion tried: True
result: 38 - (1, 'grip0')->(2, 'grip0') = success
======================= terminated 33: required answers acquired  (2.2/30.0) ===============================
branching: 38->103 (2.22/3

constrained motion tried: True
result: 36 - (1, 'grip0')->(2, 'grip0') = success
branching: 36->111 (2.38/30.0 s, steps/err: 80(843.54019165 ms)/8.69994788791e-06)
======================= terminated 13: Stop called from other agent  (2.4/30.0) ===============================
end
constrained motion tried: True
result: 13 - (1, 'grip0')->(2, 'grip0') = success
branching: 13->112 (2.43/30.0 s, steps/err: 80(836.51304245 ms)/2.97667961802e-05)
======================= terminated 27: Stop called from other agent  (2.4/30.0) ===============================
end
constrained motion tried: True
end
result: 2 - (1, 'grip0')->(2, 'grip0') = success
constrained motion tried: True
branching: 2->113 (2.51/30.0 s, steps/err: 80(644.578933716 ms)/4.97738080239e-06)
======================= terminated 3: Stop called from other agent  (2.5/30.0) ===============================
result: 69 - (2, 'grip0')->(1, 'grip0') = success
branching: 69->114 (2.51/30.0 s, steps/err: 80(669.735908508 ms)/6.46533411782e-0

In [17]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(0, 'knob')->(0, 'grip0')
(0, 'grip0')->(1, 'grip0')
(1, 'grip0')->(2, 'grip0')
(2, 'grip0')->(2, 'knob')
(2, 'knob')->(2, 'knob')


### push back

In [18]:
gscene.show_pose(crob.home_pose)
term_state = snode_schedule[-1].state
print(pscene.subject_name_list)
print(term_state.node)

['knob', 'lever_grip']
(2, 'knob')


In [19]:
goal_nodes = [(1,'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 36/36 agents
try: 0 - (2, 'knob')->(2, 'knob')
try: 0 - (2, 'knob')->(2, 'knob')
try: 0 - (2, 'knob')->(2, 'knob')
result: 0 - (2, 'knob')->(1, 'knob') = fail
try: 0 - (2, 'knob')->(2, 'grip0')
result: 0 - (2, 'knob')->(1, 'knob') = fail
result: 0 - (2, 'knob')->(1, 'knob') = fail
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
try transition motion
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
transition motion tried: True
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
result: 0 - (2, 'knob')->(2, 'grip0') = success
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'knob')
ERROR sampling parent from : (2, 'knob') / parent nodes: set([(2, 'grip0')])
try transition motion
try transition motion
try: 0 - (2, 'knob')->(2, 'knob')
transition motion trie

try: 10 - (2, 'knob')->(2, 'knob')
try: 20 - (2, 'knob')->(2, 'knob')
try: 0 - (2, 'knob')->(2, 'knob')
try: 2 - (2, 'grip0')->(2, 'grip0')
try: 20 - (2, 'knob')->(2, 'grip0')
result: 11 - (2, 'knob')->(1, 'knob') = fail
branching: 4->18 (0.67/30.0 s, steps/err: 7(92.8030014038 ms)/0.00193547850729)
try: 20 - (2, 'knob')->(2, 'grip0')
result: 20 - (2, 'knob')->(1, 'knob') = fail
result: 0 - (2, 'knob')->(1, 'knob') = fail
try: 18 - (2, 'knob')->(2, 'grip0')
result: 10 - (2, 'knob')->(1, 'knob') = fail
result: 0 - (2, 'knob')->(2, 'grip0') = success
try transition motion
try: 11 - (2, 'knob')->(2, 'knob')
try transition motion
try constrained motion
try: 16 - (2, 'grip0')->(2, 'grip0')
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
branching: 0->21 (0.72/30.0 s, steps/err: 51(466.411113739 ms)/0.000981844614581)
try constrained motion
try transition motion
try: 12 - (2, 'knob')->(2, 'knob')
result: 12 - (2, 'knob')->(1, 'knob') = fail
result: 11 - (2, 'knob')->(1, 'knob') = fa

result: 10 - (2, 'knob')->(1, 'knob') = fail
branching: 20->36 (1.04/30.0 s, steps/err: 6(116.895914078 ms)/0.00139235918776)
try transition motion
try: 14 - (2, 'knob')->(2, 'grip0')
try: 2 - (2, 'grip0')->(2, 'grip0')
try: 21 - (2, 'grip0')->(2, 'grip0')
try transition motion
try constrained motion
try constrained motion
try: 31 - (2, 'knob')->(2, 'grip0')
transition motion tried: True
joint max
result: 19 - (2, 'grip0')->(2, 'knob') = success
constrained motion tried: False
branching: 19->37 (1.1/30.0 s, steps/err: 9(71.624994278 ms)/0.00175082406999)
Motion Plan Failure
try transition motion
transition motion tried: True
result: 21 - (2, 'grip0')->(1, 'grip0') = fail
try: 37 - (2, 'knob')->(2, 'knob')
try: 18 - (2, 'knob')->(2, 'knob')
transition motion tried: True
transition motion tried: True
result: 17 - (2, 'grip0')->(2, 'knob') = success
result: 14 - (2, 'knob')->(2, 'grip0') = success
transition motion tried: True
result: 22 - (2, 'knob')->(2, 'grip0') = success
branching: 22

try: 55 - (1, 'grip0')->(1, 'knob')
end
end
result: 54 - (2, 'knob')->(1, 'knob') = fail
constrained motion tried: True
Goal reached
result: 4 - (2, 'grip0')->(1, 'grip0') = success
end
constrained motion tried: True
branching: 15->55 (1.49/30.0 s, steps/err: 80(821.505069733 ms)/2.16168044012e-05)
result: 4 - (2, 'grip0')->(1, 'grip0') = success
try: 28 - (2, 'grip0')->(2, 'grip0')
constrained motion tried: True
result: 2 - (2, 'grip0')->(1, 'grip0') = success
branching: 4->57 (1.5/30.0 s, steps/err: 80(983.791828156 ms)/2.92268849744e-06)
try constrained motion
branching: 4->58 (1.52/30.0 s, steps/err: 80(999.078989029 ms)/2.92268849744e-06)
branching: 2->59 (1.52/30.0 s, steps/err: 80(811.466217041 ms)/5.0228753776e-06)
result: 55 - (1, 'grip0')->(1, 'knob') = success
branching: 55->56 (1.53/30.0 s, steps/err: 1(6.05702400208 ms)/0)
try: 57 - (1, 'grip0')->(1, 'knob')
try transition motion
++ adding return motion to acquired answer ++
try: 50 - (2, 'knob')->(2, 'grip0')
try: 59 - (1

======================= terminated 34: Stop called from other agent  (1.9/30.0) ===============================
branching: 44->72 (1.86/30.0 s, steps/err: 2(129.889011383 ms)/0.00112631813329)
======================= terminated 6: Stop called from other agent  (1.9/30.0) ===============================
transition motion tried: True
======================= terminated 35: Stop called from other agent  (1.9/30.0) ===============================
======================= terminated 9: Stop called from other agent  (1.9/30.0) ===============================
======================= terminated 28: Stop called from other agent  (1.9/30.0) ===============================
======================= terminated 4: required answers acquired  (1.9/30.0) ===============================
Goal reached
result: 65 - (1, 'grip0')->(1, 'knob') = success
transition motion tried: True
======================= terminated 26: Stop called from other agent  (1.9/30.0) ===============================
branching: 65->73 (

end
constrained motion tried: True
end
result: 13 - (2, 'grip0')->(1, 'grip0') = success
constrained motion tried: True
branching: 13->85 (2.17/30.0 s, steps/err: 80(716.837882996 ms)/8.1243187965e-06)
result: 47 - (1, 'grip0')->(2, 'grip0') = success
======================= terminated 3: Stop called from other agent  (2.2/30.0) ===============================
end
constrained motion tried: True
end
branching: 47->86 (2.17/30.0 s, steps/err: 80(711.130857468 ms)/2.87955654637e-06)
constrained motion tried: True
result: 1 - (2, 'grip0')->(1, 'grip0') = success
result: 19 - (2, 'grip0')->(1, 'grip0') = success
======================= terminated 18: Stop called from other agent  (2.2/30.0) ===============================
branching: 19->88 (2.2/30.0 s, steps/err: 80(1013.09490204 ms)/5.70565958244e-06)
branching: 1->87 (2.19/30.0 s, steps/err: 80(745.685815811 ms)/1.05078210319e-05)
======================= terminated 5: required answers acquired  (2.2/30.0) ===============================
=

In [20]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(2, 'knob')->(2, 'grip0')
(2, 'grip0')->(1, 'grip0')
(1, 'grip0')->(1, 'knob')
(1, 'knob')->(1, 'knob')
